(PERFBLAS)=

# 5.2 Herramientas de lenguajes de programación y del sistema operativo para perfilamiento e implementaciones de BLAS

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

```{admonition} Al final de esta nota el y la lectora:
:class: tip

* 

```

En esta nota revisamos algunas herramientas de los lenguajes de programación y de los sistemas GNU/Linux para perfilamiento de código. También se revisan las operaciones de BLAS en sus diferentes niveles con ejemplos de paquetes en los lenguajes. 

Se presentan códigos y sus ejecuciones en una máquina `m5.2xlarge` de la nube de [AWS](https://aws.amazon.com/). Se sugiere se utilice la AMI --aquí colocar nombre de AMI-- de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)


Y la máquina `m5.2xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              2
Core(s) per socket:              4
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:                        7
CPU MHz:                         3353.649
BogoMIPS:                        4999.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       128 KiB
L1i cache:                       128 KiB
L2 cache:                        4 MiB
L3 cache:                        35.8 MiB
NUMA node0 CPU(s):               

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Amazon EC2
       physical id: 0
       version: 1.0
       date: 10/16/2017
       size: 64KiB
       capacity: 64KiB
       capabilities: pci edd acpi virtualmachine
  *-memory
       description: System memory
       physical id: 1
       size: 31GiB


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con *import*'s. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## Perfilamiento: medición de tiempo en *Python*

### Módulo: [time](https://docs.python.org/3/library/time.html#time.time)

Imports y funciones de apoyo:

In [3]:
import math
import time

import numpy as np
from pytest import approx
from scipy.integrate import quad

In [4]:
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res


Objetivo:

In [5]:
f=lambda x: math.exp(-x**2) #using math library
a=0
b=1
obj, err = quad(f, a, b)
print(obj)

0.7468241328124271


Medición de tiempo:

In [6]:
n=10**6
start_time = time.time()
res=Rcf(f,a,b,n)
end_time = time.time()
secs = end_time-start_time
print("Rcf tomó",secs,"segundos" )

Rcf tomó 0.3066103458404541 segundos


Prueba que se resuelve correctamente el problema:

In [7]:
print(res == approx(obj))

True


### Comando de *magic*: [%time](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time)

Nos regresa las mediciones siguientes:

* `CPU times` que contiene:

    * `user`: mide la cantidad de tiempo de los *statements* que la CPU gastó para funciones que no están relacionadas con el *kernel* del sistema.
    
    * `sys`: mide la cantidad de tiempo de los *statements* que la CPU gastó en funciones a nivel de kernel del sistema.
    
    * `total`: suma entre el `user` y `sys` para todos **todos los *cores***. 
    
* `Wall time`: mide el *wall clock* o *elapsed time* que se refiere al tiempo desde que inicia la ejecución de los *statements* hasta su finalización.

* `Out`: resultado.



```{admonition} Comentarios

* Para mediciones de tiempos que involucran cómputo en paralelo es posible que `total` exceda a `Wall time` pues es la suma de tiempos para todos los *cores*.

* Recuérdese que algunos ejemplos de funciones relacionadas con el *kernel* del sistema es el alojamiento, lectura y escritura de variables en memoria, las relacionadas con el I/O de disco o *network*.

* La diferencia entre `total` y `Wall time` da una idea de la cantidad de tiempo que se ocupó el sistema en la ejecución de *statements* que no involucran `user` ni a `sys` (por ejemplo un *statement* del tipo `sleep`) o en tareas no relacionadas con los *statements* medidos (por ejemplo si en el momento de la medición se estaban corriendo otros procesos).

```

Al medir tiempos de ejecución, **siempre** hay variación en la medición por lo que se recomienda realizar las mediciones un número repetido de veces. Para este caso el comando de *magic* `%timeit` nos ayuda. 

### Comando de *magic*: [%timeit](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)

A continuación se mide el tiempo de ejecución para la función `Rcf`. Se promedian los tiempos de $n=5$ ejecuciones y se calcula su desviación estándar. Se reptite lo anterior $r=10$ veces y se reporta el mejor resultado. 

In [8]:
%timeit -n 5 -r 10 Rcf(f,a, b,n)

301 ms ± 636 µs per loop (mean ± std. dev. of 10 runs, 5 loops each)


```{admonition} Observación
:class: tip

$ms$ es milisecond, $\mu s$ es microsecond y $ns$ es nanosecond.

```

```{admonition} Comentarios

* `%timeit` se recomienda usar para secciones de código pequeñas. Si se observa una variación en las mediciones de tiempo entre distintas repeticiones entonces hay que realizar más repeticiones hasta tener un resultado estable.

* `%timeit` desabilita temporalmente el *garbage collector* de *Python* (esto es, no habrá desalojamiento en memoria de objetos de Python que no se utilicen). Si el *garbage collector* es invocado en tus *statements*, esto puede ser una razón de posibles diferencias que se obtengan en las mediciones de tiempo.


```

## Perfilamiento: medición de uso de CPU en *Python*

### [line_profiler](https://pypi.org/project/line-profiler/)

### [CProfile](https://docs.python.org/2/library/profile.html) 

*CProfile* es `built-in` en la *standard-library* de Python.

## Perfilamiento: medición de uso de memoria en *Python*

### [memory_profiler](https://pypi.org/project/memory-profiler/)

### [heapy](https://pypi.org/project/guppy/)

## BLAS

### OpenBLAS y *NumPy*

### OpenBLAS y librerías de cómputo matricial de *R*

## Perfilamiento: medición de tiempo en el sistema operativo Ubuntu 20.04

### [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html)

Ver también: [Wikipedia: Time_Unix](https://en.wikipedia.org/wiki/Time_(Unix))


In [9]:
%%file Rcf.py
import math

from pytest import approx
from scipy.integrate import quad
def Rcf(f,a,b,n):
    """
    Compute numerical approximation using rectangle or mid-point
    method in an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for
    i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
    
        f (function): function expression of integrand.
        
        a (float): left point of interval.
        
        b (float): right point of interval.
        
        n (float): number of subintervals.
        
    Returns:
    
        sum_res (float): numerical approximation to integral
            of f in the interval a,b
    """
    h_hat=(b-a)/n
    nodes=[a+(i+1/2)*h_hat for i in range(0,n)]
    sum_res=0
    for node in nodes:
        sum_res=sum_res+f(node)
    return h_hat*sum_res
if __name__=="__main__": #añadimos este bloque para ejecución de la función Rcf
    a=0
    b=1
    f=lambda x: math.exp(-x**2)
    obj, err = quad(f, a, b)
    print("objetivo: {:0.6e}".format(obj))
    n=10**6
    res=Rcf(f,a,b,n)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))

Writing Rcf.py


In [10]:
%%bash
/usr/bin/time -p python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


real 0.64
user 0.91
sys 0.48


In [11]:
%%bash
/usr/bin/time -v python3 Rcf.py

objetivo: 7.468241e-01
aproximación: 7.468241e-01
True


	Command being timed: "python3 Rcf.py"
	User time (seconds): 0.90
	System time (seconds): 0.47
	Percent of CPU this job got: 219%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.63
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 97272
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 20709
	Voluntary context switches: 13
	Involuntary context switches: 14
	Swaps: 0
	File system inputs: 0
	File system outputs: 0
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


```{admonition} Comentario

Una breve explicación del *output* se puede encontrar [/usr/bin/time](http://manpages.ubuntu.com/manpages/focal/man1/time.1.html). Por ejemplo, para el caso de `Major (requiring I/O)` nos interesa que sea $0$ pues indica que el sistema operativo tiene que cargar páginas de datos del disco pues tales datos ya no residen en RAM (por alguna razón).

```

```{admonition} Observación
:class: tip

La medición `real` que regresa `/usr/bin/time` es el equivalente al `Wall time` de `%time`.


```

## Perfilamiento: medición de uso de CPU en el sistema operativo GNU/Linux

### [perf](https://github.com/torvalds/linux/tree/master/tools/perf)

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Referencias:**

1. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

2. E. Anderson, Z. Bai, C. Bischof, L. S. Blackford, J. Demmel, J. Dongarra, J. Du Croz,
A. Greenbaum, S. Hammarling, A. Mckenney and D. Sorensen, LAPACK Users Guide, Society for Industrial and Applied Mathematics, Philadelphia, PA, third ed., 1999.

